# Backtesting Examples and Guides

In [1]:
import pandas as pd
import vectorbtpro as vbt
import numpy as np

In [2]:
vbt.settings.wrapping ["freq"]                = "1m"
vbt.settings.portfolio['init_cash']           = 10000

In [3]:
import os
import sys

sys.path.append(os.getcwd())

from dataframes_merger_factory import DataFrameMergerFactory
from dataframes_merger import DataFrameMergerType
from parameter_optimization import DataFrameFormat
from parameter_optimization_factory import VbtBackTestProcessorType, VbtBackTestProcessorFactory
from settings_and_params import extract_prediction_window_size
from lstm_analysis_utils import process_pickle_files
from multiple_models_backtesting_factory import MultiModelBacktestMethod, MultiModelBacktestFactory

# Running backtest on just one RID

### Old format - input is a directory containing pickle files

In [ ]:
pickle_files_path               = "../data/RID0047_LSTM_pw75_lb250_bt1000_mem10000"

model_name                      = pickle_files_path.split('/')[-1]
prediction_window               = extract_prediction_window_size(model_name)

- must use DataFrameFormat.SINGLE
- VbtBackTestProcessorType can be any type that your system can handle
- Recommended value is VbtBackTestProcessorType.WITH_MEMORY_CONSTRAINT_TWO_LOOPS unless you have more than 64 GB of RAM

In [ ]:
df1 = process_pickle_files(pickle_files_path, prediction_window)
df1_result = VbtBackTestProcessorFactory.create(VbtBackTestProcessorType.WITH_MEMORY_CONSTRAINT_TWO_LOOPS, df1, prediction_window, DataFrameFormat.SINGLE).run_backtest()

### New format - input is the CSV file containing the dataframe dumped by calling process_pickle_files() first
Or it could be the files downloaded from our Google shared drive

In [ ]:
DATAFRAME_CSV_FILES    = ["../results/RID0047_LSTM_pw75_lb250_bt1000_mem10000.csv"]

model_name             = DATAFRAME_CSV_FILES[0].split("/")[-1].split(".")[0]
prediction_window      = extract_prediction_window_size(model_name)

- Must use DataFrameFormat.MERGED
- VbtBackTestProcessorType can be any type that your system can handle
- Recommended value is VbtBackTestProcessorType.WITH_MEMORY_CONSTRAINT_TWO_LOOPS unless you have more than 64 GB of RAM

In [ ]:
df2 = DataFrameMergerFactory.create(DataFrameMergerType.INTERSECTION).process(DATAFRAME_CSV_FILES)
df2_result = VbtBackTestProcessorFactory.create(VbtBackTestProcessorType.WITH_MEMORY_CONSTRAINT_TWO_LOOPS, df2, prediction_window, DataFrameFormat.MERGED).run_backtest()

# Running backtest on multiple RIDs

### Using average method

In [ ]:
MISMATCHING_DATAFRAME_CSV_FILES = [  "../results/RID0029_LSTM_pw38_lb250_bt2000_mem6000.csv"
                                   , "../results/RID0045_LSTM_pw38_lb5000_bt2000_mem10000.csv"
                                   , "../results/RID0046_LSTM_pw38_lb250_bt2000_mem6000.csv"]

The merger type can either be INTERSECTION or UNION

In [ ]:
merger = DataFrameMergerFactory.create(DataFrameMergerType.INTERSECTION)

Running the back test is one line of code

In [ ]:
df, result = MultiModelBacktestFactory.create(MultiModelBacktestMethod.AVERAGE, merger, MISMATCHING_DATAFRAME_CSV_FILES).run()

### Using majority method

In [4]:
MODEL_NAMES = ["RID0029_LSTM_pw38_lb250_bt2000_mem6000", "RID0045_LSTM_pw38_lb5000_bt2000_mem10000"]

In [5]:
merger = DataFrameMergerFactory.create(DataFrameMergerType.INTERSECTION)

In [6]:
df, result = MultiModelBacktestFactory.create(MultiModelBacktestMethod.MAJORITY, merger, MODEL_NAMES).run()